In [2]:
from telegram_models import TelegramExport
from pprint import pprint

In [3]:
chats = TelegramExport.parse_file('result.json').chats

In [4]:
chat = next(chat for chat in chats if chat.name == 'Сергей')

pprint(chat.dict(), indent=2, depth=1)

{ 'id_': 254968821,
  'messages': [...],
  'name': 'Сергей',
  'type_': <ChatType.Chat: 'personal_chat'>}


In [5]:
chunks = []

# до определенного промежутка мы считаем, что все сообщения относятся к одному под-диалогу
# после определенного промежутка мы считаем, что все сообщения относятся к другому под-диалогу
# между этими промежутками мы доверяем это решение нейросети которая вычисляет specifity ответа на сообщение
# причем делаем с определенным процентом доверенности
# (4 часа -> думаем, что сообщения относятся к одному под-диалогу, но если нейросеть думает сильно иначе, то к разным)
# (8 часов -> наоборот, с большей вероятностью думаем что сообщения из разных диалогов, но если нейросеть нас переубедит, то из одного)

chunks.append(chat.messages[0])

def get_relevance(message):
    # TODO: use NN to get relevance
    from random import random
    return random()

def is_relevant(time_delta):
    from math import cos, pi
    start = 3
    end = 9
    up_bound = 0.95
    down_bound = 0.05

    if time_delta < start:
        return down_bound
    elif time_delta > end:
        return up_bound
    else:
        d = up_bound - down_bound
        v = cos(pi * (time_delta - start) / (end - start))
        v = d * (1 - v) / 2 + down_bound
        return v
    

for i in range(1, 10): # len(chat.messages)):
    delta = chat.messages[i].date - chat.messages[i - 1].date
    delta_hours = delta.total_seconds() / 3600
    print(round(delta_hours, 2), is_relevant(delta_hours))


0.17 0.05
0.9 0.05
2.6 0.05
4.76 0.22740935865127454
0.0 0.05
0.18 0.05
20.99 0.95
0.0 0.05
0.46 0.05


In [6]:
import altair as alt
import pandas as pd

df = pd.DataFrame({'x': [i / 20 for i in range(240)], 'y': [is_relevant(i /20) for i in range(240)]})

# add tooltip to graph
alt.Chart(df).mark_line().encode(
    x='x',
    y='y',
    tooltip=['x', 'y']
)

alt.Chart(...)